<a href="https://colab.research.google.com/github/NavePnow/Auto-Hashtag-of-Social-Posts/blob/master/Train_model_and_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Package Installation

In [2]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [3]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6593 (delta 0), reused 0 (delta 0), pack-reused 6589
Receiving objects: 100% (6593/6593), 13.70 MiB | 17.79 MiB/s, done.
Resolving deltas: 100% (4383/4383), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-36or0ixj
Created temporary directory: /tmp/pip-req-tracker-g1n_qzf1
Created requirements tracker '/tmp/pip-req-tracker-g1n_qzf1'
Created temporary directory: /tmp/pip-install-7vauct7k
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-se3opjmx
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-g1n_qzf1'
    Running setup.py (path:/tmp/pi

In [3]:
!pip install fast-bert
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 573kB 13.2MB/s 
     |████████████████████████████████| 3.8MB 24.3MB/s 
     |████████████████████████████████| 204kB 55.4MB/s 
     |████████████████████████████████| 112kB 57.0MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
     |████████████████████████████████| 1.0MB 51.6MB/s 
     |████████████████████████████████| 552kB 58.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=0976a6709bd295c62f71b64f9901f67fd346e4ec3af4ab990b567da1c3fa6dc9
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.7.0 which is incompatible.
     |████████████████████████████████| 133kB 9.6MB/s 


# Environment Setting


In [0]:
import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 512,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [0]:
from fast_bert.learner_cls import BertLearner
from fast_bert.data_cls import BertDataBunch
from fast_bert.metrics import accuracy
from fast_bert.metrics import accuracy_multilabel

# Load Dataset

In [6]:
!git clone https://github.com/NavePnow/multi-label-classification.git

Cloning into 'multi-label-classification'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 46 (delta 9), reused 45 (delta 8), pack-reused 0
Unpacking objects: 100% (46/46), done.


# Train and Save model

In [0]:
DATA_PATH = Path('/content/multi-label-classification/data/')     
LABEL_PATH = Path('/content/multi-label-classification/labels/')  
OUTPUT_DIR = Path('/content/multi-label-classification/')
BERT_PRETRAINED_MODEL = "bert-base-uncased"

args["do_lower_case"] = True
args["train_batch_size"] = 16
args["learning_rate"] = 6e-5
args["max_seq_length"] = 140
args["fp16"] = True

In [8]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_MODEL, 
                                          do_lower_case=args['do_lower_case'])

04/24/2020 15:32:36 - INFO - pytorch_pretrained_bert.modeling -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
04/24/2020 15:32:37 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmp85bncc2i


100%|██████████| 231508/231508 [00:00<00:00, 902316.19B/s]

04/24/2020 15:32:37 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmp85bncc2i to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/24/2020 15:32:37 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/24/2020 15:32:37 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmp85bncc2i
04/24/2020 15:32:37 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
label_cols = ["friends", "pets", "tuesdaymotivation", "funny", "contest", "giveaway", "ootd", "win", "merrychristmas", "competition", "fridayfeeling", "traveltuesday", "happybirthday", "wcw", "goals", "fitness", "vegan", "movies", "running", "thankful", "science", "blessed", "influencer", "metoo"]

In [0]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH, tokenizer='bert-base-uncased', train_file='train.csv', val_file='valid.csv',
                          label_file="labels.csv",
                          text_col="tweet", label_col=label_cols,
                          batch_size_per_gpu=args['train_batch_size'], max_seq_length=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)

In [0]:
metrics = [{'name': 'accuracy', 'function': accuracy_multilabel}]

In [9]:
logger.info(args)

04/24/2020 15:30:32 - INFO - root -   {'run_text': 'my_test', 'max_seq_length': 140, 'do_lower_case': True, 'train_batch_size': 16, 'learning_rate': 6e-05, 'num_train_epochs': 12.0, 'warmup_proportion': 0.002, 'local_rank': -1, 'gradient_accumulation_steps': 1, 'fp16': True, 'loss_scale': 128}


In [0]:
import logging
device_cuda = torch.device("cuda")
logger = logging.getLogger()
learner = BertLearner.from_pretrained_model(databunch, BERT_PRETRAINED_MODEL, metrics=metrics, device=device_cuda, logger=logger, output_dir=OUTPUT_DIR, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)

In [0]:
learner.fit(4, lr=args['learning_rate'], schedule_type="warmup_linear")

In [0]:
learner.save_model()

# Make prediction

In [0]:
from fast_bert.prediction import BertClassificationPredictor
predictor = BertClassificationPredictor(
    model_path="/content/multi-label-classification/model_out",
    label_path="/content/multi-label-classification/labels/",  # location for labels.csv file
    multi_label=True,
    # model_type='xlnet',
    do_lower_case=True)
prediction = predictor.predict("i like running")
print(prediction)